In [1]:
import sys
import os
from baselines import logger, bench
from baselines.common import set_global_seeds
from baselines.common.cmd_util import make_atari_env, atari_arg_parser
from baselines.common.atari_wrappers import wrap_deepmind, make_atari
from baselines.common.vec_env.vec_frame_stack import VecFrameStack
from baselines.common.vec_env.dummy_vec_env import DummyVecEnv
from baselines.common.vec_env.subproc_vec_env import SubprocVecEnv
from baselines.common.vec_env.vec_normalize import VecNormalize, VecNormalizeTest
from baselines.ppo2 import ppo2
#from baselines.ppo2.policies import CnnPolicy, LstmPolicy, LnLstmPolicy, MlpPolicy
import multiprocessing
import tensorflow as tf
import pandas as pd
import numpy as np
import gym

from src.common.environments import ESTradingEnv

import matplotlib as mpl
%matplotlib

/home/ubuntu/src/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using matplotlib backend: Qt5Agg


/home/ubuntu/src/anaconda3/envs/tensorflow/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:106: MatplotlibDeprecationWarning: The finance module has been deprecated in mpl 2.0 and will be removed in mpl 2.2. Please use the module mpl_finance instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [2]:
#Register trading environment with OpenAI Gym
from gym.envs.registration import register
register(id='tradingenv-v0',
         entry_point='src.common.environments:ESTradingEnv_v2',)

In [3]:
from baselines.a2c.utils import fc, conv_to_fc, lstm, batch_to_seq, seq_to_batch
from baselines.common.distributions import make_pdtype
from baselines.common.input import observation_input

def ortho_init(scale=1.0):
    def _ortho_init(shape, dtype, partition_info=None):
        #lasagne ortho init for tf
        shape = tuple(shape)
        if len(shape) == 2:
            flat_shape = shape
        elif len(shape) > 2: # assumes NHWC
            flat_shape = (np.prod(shape[:-1]), shape[-1])
        else:
            raise NotImplementedError
        a = np.random.normal(0.0, 1.0, flat_shape)
        u, _, v = np.linalg.svd(a, full_matrices=False)
        q = u if u.shape == flat_shape else v # pick the one with the correct shape
        q = q.reshape(shape)
        return (scale * q[:shape[0], :shape[1]]).astype(np.float32)
    return _ortho_init

def conv(x, scope, *, nf, rf, stride, pad='VALID', init_scale=1.0, one_dim_bias=False):    
    channel_ax = 2
    strides = [1, stride, 1]
    bshape = [1, 1, nf]
    
    nin = x.get_shape()[channel_ax].value
    wshape = [rf, nin, nf]
    with tf.variable_scope(scope):
        w = tf.get_variable("w", wshape, initializer=ortho_init(init_scale))
        b = tf.get_variable("b", bshape, initializer=tf.constant_initializer(0.0))
        return b + tf.nn.conv1d(x, w, stride=stride, padding=pad)


def cnn(X, is_training, **conv_kwargs):
    activ = tf.nn.relu
    h = activ(conv(X, 'c1', nf=16, rf=4, stride=1, init_scale=np.sqrt(2),
                   **conv_kwargs))
    #h = tf.layers.dropout(h, rate=0.5, training=is_training)
    h2 = activ(conv(h, 'c2', nf=8, rf=4, stride=1, init_scale=np.sqrt(2), **conv_kwargs))
    #h2 = tf.layers.dropout(h2, rate=0.5, training=is_training)
    h3 = conv_to_fc(h2)
    return activ(fc(h3, 'fc1', nh=64, init_scale=np.sqrt(2)))


class CnnPolicy(object):

    def __init__(self, sess, ob_space, ac_space, nbatch, nsteps, reuse=False, **conv_kwargs): #pylint: disable=W0613
        if nsteps == 1:
            self._is_training = False
        else:
            self._is_training = True
            
        self.pdtype = make_pdtype(ac_space)
        X, processed_x = observation_input(ob_space, nbatch)
        with tf.variable_scope("model", reuse=reuse):
            h = cnn(processed_x, self._is_training, **conv_kwargs)
            #activ = tf.nn.relu
            #print(processed_x.get_shape())
            #h = activ(fc(processed_x, 'fc1', nh=512, init_scale=np.sqrt(2)))
            vf = fc(h, 'v', 1)[:,0]
            self.pd, self.pi = self.pdtype.pdfromlatent(h, init_scale=0.01)

        a0 = self.pd.sample()
        neglogp0 = self.pd.neglogp(a0)
        self.initial_state = None

        def step(ob, *_args, **_kwargs):
            a, v, neglogp = sess.run([a0, vf, neglogp0], {X:ob})
            return a, v, self.initial_state, neglogp

        def value(ob, *_args, **_kwargs):
            return sess.run(vf, {X:ob})

        self.X = X
        self.vf = vf
        self.step = step
        self.value = value
        
class LstmPolicy(object):

    def __init__(self, sess, ob_space, ac_space, nbatch, nsteps, nlstm=32, reuse=False):
        if nsteps == 1:
            self._is_training = False
        else:
            self._is_training = True
            
        nenv = nbatch // nsteps
        self.pdtype = make_pdtype(ac_space)
        X, processed_x = observation_input(ob_space, nbatch)

        M = tf.placeholder(tf.float32, [nbatch]) #mask (done t-1)
        S = tf.placeholder(tf.float32, [nenv, nlstm*2]) #states
        with tf.variable_scope("model", reuse=reuse):
            h = cnn(X, self._is_training)
            #h = processed_x
            #h = tf.layers.dropout(h, rate=0.5, training=self._is_training)
            xs = batch_to_seq(h, nenv, nsteps)
            ms = batch_to_seq(M, nenv, nsteps)
            h5, snew = lstm(xs, ms, S, 'lstm1', nh=nlstm)
            #h5 = tf.layers.dropout(h5, rate=0.5, training=self._is_training)
            h5 = seq_to_batch(h5)
            vf = fc(h5, 'v', 1)
            self.pd, self.pi = self.pdtype.pdfromlatent(h5)

        v0 = vf[:, 0]
        a0 = self.pd.sample()
        neglogp0 = self.pd.neglogp(a0)
        self.initial_state = np.zeros((nenv, nlstm*2), dtype=np.float32)

        def step(ob, state, mask):
            a, v, s, neglogp = sess.run([a0, v0, snew, neglogp0], {X:ob, S:state, M:mask})
            return a, v, s, neglogp

        def value(ob, state, mask):
            return sess.run(v0, {X:ob, S:state, M:mask})

        self.X = X
        self.M = M
        self.S = S
        self.vf = vf
        self.step = step
        self.value = value

In [4]:
def train(env_id, num_timesteps, seed, policy):

    ncpu = multiprocessing.cpu_count()
    if sys.platform == 'darwin': ncpu //= 2
    config = tf.ConfigProto(allow_soft_placement=True,
                            intra_op_parallelism_threads=ncpu,
                            inter_op_parallelism_threads=ncpu)
    config.gpu_options.allow_growth = True #pylint: disable=E1101
    tf.Session(config=config).__enter__()
    
    def make_env(rank):
        def _thunk():
            env = gym.make(env_id)
            env.seed(seed + rank)
            env = bench.Monitor(env, logger.get_dir() and os.path.join(logger.get_dir(), str(rank)))
            return env
        return _thunk
    

    #env = DummyVecEnv([make_env])
    env = SubprocVecEnv([make_env(i) for i in range(32)])
    env = VecNormalize(env, ob=True)
    set_global_seeds(seed)

    policy = LstmPolicy
    ppo2.learn(policy=policy, env=env, nsteps=256, nminibatches=32,
        lam=0.95, gamma=0.99, noptepochs=20, log_interval=1,
        ent_coef=.01,
        lr=lambda f: f * 2.5e-3,
        cliprange=0.1,
        total_timesteps=num_timesteps)

In [ ]:
logger.configure()

train('tradingenv-v0', num_timesteps=1e7, seed=0, policy='cnn')

Logging to /tmp/openai-2018-08-23-09-38-32-256117
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
-------------------------------------
| approxkl           | 0.011441799  |
| clipfrac           | 0.4671448    |
| eplenmean          | 111          |
| eprewmean          | -7.79e+03    |
| explained_variance | 0.0407       |
| fps                | 95           |
| nupdates           | 1            |
| policy_entropy     | 1.0606921    |
| policy_loss        | -0.029196804 |
| serial_timesteps   | 256          |
| time_elapsed       | 86           |
| total_timesteps    | 8192         |
| value_loss         | 0.4977464    |
-------------------------------------
Saving to /tmp/openai-2018-08-23-09-38-32-

------------------------------------
| approxkl           | 0.23909214  |
| clipfrac           | 0.73638916  |
| eplenmean          | 174         |
| eprewmean          | -7.61e+03   |
| explained_variance | 0.00378     |
| fps                | 116         |
| nupdates           | 14          |
| policy_entropy     | 0.70232886  |
| policy_loss        | -0.04372043 |
| serial_timesteps   | 3584        |
| time_elapsed       | 998         |
| total_timesteps    | 114688      |
| value_loss         | 0.0414134   |
------------------------------------
------------------------------------
| approxkl           | 0.25558668  |
| clipfrac           | 0.7400757   |
| eplenmean          | 182         |
| eprewmean          | -7.43e+03   |
| explained_variance | 0.0842      |
| fps                | 117         |
| nupdates           | 15          |
| policy_entropy     | 0.6939071   |
| policy_loss        | -0.04488061 |
| serial_timesteps   | 3840        |
| time_elapsed       | 1.07e+03    |
|

-------------------------------------
| approxkl           | 0.37088498   |
| clipfrac           | 0.5581726    |
| eplenmean          | 222          |
| eprewmean          | -3.62e+03    |
| explained_variance | -0.00105     |
| fps                | 115          |
| nupdates           | 29           |
| policy_entropy     | 0.48334938   |
| policy_loss        | -0.027583372 |
| serial_timesteps   | 7424         |
| time_elapsed       | 2.05e+03     |
| total_timesteps    | 237568       |
| value_loss         | 0.008417767  |
-------------------------------------
-------------------------------------
| approxkl           | 0.3326018    |
| clipfrac           | 0.5780029    |
| eplenmean          | 224          |
| eprewmean          | -3.25e+03    |
| explained_variance | 0.0267       |
| fps                | 114          |
| nupdates           | 30           |
| policy_entropy     | 0.50313294   |
| policy_loss        | -0.028612971 |
| serial_timesteps   | 7680         |
| time_elaps

-------------------------------------
| approxkl           | 0.33654916   |
| clipfrac           | 0.5216858    |
| eplenmean          | 191          |
| eprewmean          | -1.07e+03    |
| explained_variance | -0.0363      |
| fps                | 114          |
| nupdates           | 44           |
| policy_entropy     | 0.4288748    |
| policy_loss        | -0.021984074 |
| serial_timesteps   | 11264        |
| time_elapsed       | 3.12e+03     |
| total_timesteps    | 360448       |
| value_loss         | 0.0016876959 |
-------------------------------------
-------------------------------------
| approxkl           | 0.34646285   |
| clipfrac           | 0.5214294    |
| eplenmean          | 185          |
| eprewmean          | -1.13e+03    |
| explained_variance | -0.00877     |
| fps                | 116          |
| nupdates           | 45           |
| policy_entropy     | 0.4236408    |
| policy_loss        | -0.019619785 |
| serial_timesteps   | 11520        |
| time_elaps

--------------------------------------
| approxkl           | 0.30827639    |
| clipfrac           | 0.4874939     |
| eplenmean          | 189           |
| eprewmean          | -710          |
| explained_variance | -0.0334       |
| fps                | 113           |
| nupdates           | 59            |
| policy_entropy     | 0.41527873    |
| policy_loss        | -0.01925296   |
| serial_timesteps   | 15104         |
| time_elapsed       | 4.19e+03      |
| total_timesteps    | 483328        |
| value_loss         | 0.00066393265 |
--------------------------------------
--------------------------------------
| approxkl           | 0.35026246    |
| clipfrac           | 0.48726195    |
| eplenmean          | 180           |
| eprewmean          | -714          |
| explained_variance | -0.00929      |
| fps                | 114           |
| nupdates           | 60            |
| policy_entropy     | 0.40082288    |
| policy_loss        | -0.01931334   |
| serial_timesteps   | 15

...

--------------------------------------
| approxkl           | 1.2263047e-05 |
| clipfrac           | 0.0004272461  |
| eplenmean          | 1.62e+03      |
| eprewmean          | -17.4         |
| explained_variance | -3.1e-06      |
| fps                | 116           |
| nupdates           | 703           |
| policy_entropy     | 0.007895035   |
| policy_loss        | -7.604631e-05 |
| serial_timesteps   | 179968        |
| time_elapsed       | 4.93e+04      |
| total_timesteps    | 5758976       |
| value_loss         | 1.4730192e-06 |
--------------------------------------
---------------------------------------
| approxkl           | 1.8864583e-05  |
| clipfrac           | 0.0008972168   |
| eplenmean          | 1.62e+03       |
| eprewmean          | -15.7          |
| explained_variance | -0.000415      |
| fps                | 117            |
| nupdates           | 704            |
| policy_entropy     | 0.0074727847   |
| policy_loss        | -0.00018788931 |
| serial_timest

---------------------------------------
| approxkl           | 9.024052e-06   |
| clipfrac           | 0.0008911133   |
| eplenmean          | 1.62e+03       |
| eprewmean          | -8.1           |
| explained_variance | 0.00785        |
| fps                | 112            |
| nupdates           | 717            |
| policy_entropy     | 0.009663525    |
| policy_loss        | -0.00017798762 |
| serial_timesteps   | 183552         |
| time_elapsed       | 5.03e+04       |
| total_timesteps    | 5873664        |
| value_loss         | 5.0945596e-06  |
---------------------------------------
--------------------------------------
| approxkl           | 1.2391925e-05 |
| clipfrac           | 0.00084838865 |
| eplenmean          | 1.62e+03      |
| eprewmean          | -8.64         |
| explained_variance | -0.00156      |
| fps                | 109           |
| nupdates           | 718           |
| policy_entropy     | 0.00904512    |
| policy_loss        | 1.3376062e-06 |
| serial_t

---------------------------------------
| approxkl           | 1.1899794e-05  |
| clipfrac           | 0.00095214846  |
| eplenmean          | 1.62e+03       |
| eprewmean          | -10.2          |
| explained_variance | 0.016          |
| fps                | 115            |
| nupdates           | 731            |
| policy_entropy     | 0.010772943    |
| policy_loss        | -7.9808924e-05 |
| serial_timesteps   | 187136         |
| time_elapsed       | 5.13e+04       |
| total_timesteps    | 5988352        |
| value_loss         | 9.621978e-06   |
---------------------------------------
---------------------------------------
| approxkl           | 6.926316e-06   |
| clipfrac           | 0.00057373045  |
| eplenmean          | 1.62e+03       |
| eprewmean          | -9.18          |
| explained_variance | -8.82e-06      |
| fps                | 115            |
| nupdates           | 732            |
| policy_entropy     | 0.010135362    |
| policy_loss        | -1.7895782e-05 |


--------------------------------------
| approxkl           | 1.2310731e-05 |
| clipfrac           | 0.0006896973  |
| eplenmean          | 1.62e+03      |
| eprewmean          | -12.9         |
| explained_variance | -0.000894     |
| fps                | 120           |
| nupdates           | 745           |
| policy_entropy     | 0.00935677    |
| policy_loss        | -9.811347e-05 |
| serial_timesteps   | 190720        |
| time_elapsed       | 5.23e+04      |
| total_timesteps    | 6103040       |
| value_loss         | 6.5643444e-06 |
--------------------------------------
---------------------------------------
| approxkl           | 1.4833482e-05  |
| clipfrac           | 0.0013549805   |
| eplenmean          | 1.62e+03       |
| eprewmean          | -13            |
| explained_variance | -0.00246       |
| fps                | 118            |
| nupdates           | 746            |
| policy_entropy     | 0.008550572    |
| policy_loss        | -0.00018156474 |
| serial_timest

--------------------------------------
| approxkl           | 1.2943774e-05 |
| clipfrac           | 0.0011047364  |
| eplenmean          | 1.62e+03      |
| eprewmean          | -14.8         |
| explained_variance | 0.00638       |
| fps                | 122           |
| nupdates           | 759           |
| policy_entropy     | 0.009702372   |
| policy_loss        | -7.633456e-05 |
| serial_timesteps   | 194304        |
| time_elapsed       | 5.33e+04      |
| total_timesteps    | 6217728       |
| value_loss         | 2.0250753e-05 |
--------------------------------------
---------------------------------------
| approxkl           | 7.779485e-06   |
| clipfrac           | 0.0006469727   |
| eplenmean          | 1.62e+03       |
| eprewmean          | -14.3          |
| explained_variance | -0.0111        |
| fps                | 119            |
| nupdates           | 760            |
| policy_entropy     | 0.009740356    |
| policy_loss        | -0.00010185887 |
| serial_timest

In [6]:
load_path = '/tmp/openai-2018-05-29-15-08-04-310254/checkpoints/00100'
logger.get_dir()

'/tmp/openai-2018-05-29-15-08-04-310254'

In [4]:
def test(env_id, load_path, seed):

    ncpu = multiprocessing.cpu_count()
    if sys.platform == 'darwin': ncpu //= 2
    config = tf.ConfigProto(allow_soft_placement=True,
                            intra_op_parallelism_threads=ncpu,
                            inter_op_parallelism_threads=ncpu)
    config.gpu_options.allow_growth = True #pylint: disable=E1101
    tf.Session(config=config).__enter__()
    
    def make_env():
        env = gym.make(env_id)
        return env
    
    
    env = DummyVecEnv([make_env])
    running_mean = np.load('{}/mean.npy'.format(load_path))
    running_var = np.load('{}/var.npy'.format(load_path))
    env = VecNormalizeTest(env, running_mean, running_var)
    set_global_seeds(seed)

    policy = LstmPolicy
    ob_space = env.venv.observation_space
    ac_space = env.venv.action_space
    nbatch_act = 1 * 1
    nbatch_train = 32 * 8
    nsteps = 256
    ent_coef = .01
    vf_coef = 0.5
    max_grad_norm = 0.5
    make_model = lambda : ppo2.Model(policy=policy, ob_space=ob_space, ac_space=ac_space, 
                                nbatch_act=nbatch_act, nbatch_train=nbatch_train, nsteps=nsteps, 
                                ent_coef=ent_coef, vf_coef=vf_coef, max_grad_norm=max_grad_norm)
    model = make_model()
    model.load(load_path + '/checkpoints/01200')
    
    obs = env.reset()
    states = model.initial_state
    dones = [False for _ in range(1)]
    while True:
        actions, _, states, _, = model.step(obs, states, dones)
        obs[:], _, dones, _, = env.step(actions)
        env.venv.render()

In [ ]:
load_path = '../data/processed'
load_path = '/tmp/openai-2018-06-22-12-45-22-090687'
test('tradingenv-v0', load_path, seed=0)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [5]:
def make_env():
        env = gym.make('tradingenv-v0')
        return env
env = DummyVecEnv([make_env])
load_path = '/tmp/openai-2018-05-29-15-08-04-310254'
running_mean = np.load('{}/mean.npy'.format(load_path))
running_var = np.load('{}/var.npy'.format(load_path))
env = VecNormalizeTest(env, running_mean, running_var)

In [3]:
import gym
e = gym.make('tradingenv-v5')

In [13]:
e.step(1)

(array([[ 2.44675000e+03,  2.44800000e+03,  2.44675000e+03,
          2.44725000e+03, -8.43391446e-01, -5.37299608e-01,
          0.00000000e+00],
        [ 2.44725000e+03,  2.44800000e+03,  2.44675000e+03,
          2.44700000e+03, -8.54911871e-01, -5.18773258e-01,
          0.00000000e+00],
        [ 2.44700000e+03,  2.44775000e+03,  2.44675000e+03,
          2.44700000e+03, -8.66025404e-01, -5.00000000e-01,
          0.00000000e+00],
        [ 2.44725000e+03,  2.44875000e+03,  2.44700000e+03,
          2.44775000e+03, -8.76726756e-01, -4.80988769e-01,
          0.00000000e+00],
        [ 2.44800000e+03,  2.44925000e+03,  2.44775000e+03,
          2.44850000e+03, -8.87010833e-01, -4.61748613e-01,
          0.00000000e+00],
        [ 2.44850000e+03,  2.44950000e+03,  2.44850000e+03,
          2.44900000e+03, -8.96872742e-01, -4.42288690e-01,
          0.00000000e+00],
        [ 2.44900000e+03,  2.44950000e+03,  2.44850000e+03,
          2.44850000e+03, -9.06307787e-01, -4.22618262e-01